# GraphRAG (local Search)

### Installation of Libraries

In [ ]:
# !pip install graphrag
# !pip install PyMuPDF

In [2]:
import fitz  # PyMuPDF
import os
import shutil
import subprocess

In [2]:
# !mkdir -p ./ragtest/input

### Deleteing already existing files inside input directory

In [3]:
# Directory path
directory = r'C:\Users\15011\Documents\Projects\GraphRAG\ragtest\input'

# Get all files in the directory
files = os.listdir(directory)

# Iterate over all files and delete each one
for file in files:
    file_path = os.path.join(directory, file)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted {file_path}")
        else:
            print(f"{file_path} is not a file.")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

Deleted C:\Users\15011\Documents\Projects\GraphRAG\ragtest\input\38022122.txt


### Settting Up the Workspace Variables

In [ ]:
# !python -m graphrag.index --init --root ./ragtest

### Importing the data (.txt files)

In [4]:
# !curl https://www.gutenberg.org/cache/epub/24022/pg24022.txt > ./ragtest/input/book.txt

In [5]:
# Define the directories
pdf_dir = r'C:\Users\15011\Documents\Projects\GraphRAG\ragtest\pdfs'
output_dir = r'C:\Users\15011\Documents\Projects\GraphRAG\ragtest\input'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process each PDF file in the directory
for pdf_file_name in os.listdir(pdf_dir):
    if pdf_file_name.endswith('.pdf'):
        pdf_path = os.path.join(pdf_dir, pdf_file_name)
        txt_file_name = os.path.splitext(pdf_file_name)[0] + '.txt'
        output_path = os.path.join(output_dir, txt_file_name)

        # Open the PDF file
        pdf_document = fitz.open(pdf_path)

        # Extract text from each page
        full_text = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            full_text += page.get_text()

        # Save the extracted text to a .txt file
        with open(output_path, 'w', encoding='utf-8') as text_file:
            text_file.write(full_text)

        print(f'Text extracted and saved to {output_path}')


Text extracted and saved to C:\Users\15011\Documents\Projects\GraphRAG\ragtest\input\38022122.txt


### Deleting the previous available data at output folder

In [7]:
# Define the directory to be deleted
dir_path = r'C:\Users\15011\Documents\Projects\GraphRAG\ragtest\output'

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f'Directory {dir_path} has been deleted successfully.')
else:
    print(f'Directory {dir_path} does not exist.')


Directory C:\Users\15011\Documents\Projects\GraphRAG\ragtest\output has been deleted successfully.


### Running the Indexing pipeline

In [8]:
!python -m graphrag.index --root ./ragtest

⠋ GraphRAG Indexer 
🚀 Reading settings from ragtest\settings.yaml
⠋ GraphRAG Indexer 
⠋ GraphRAG Indexer 
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ----- 100% 0:00:… 0:00:…
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ----- 100% 0:00:… 0:00:…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ----- 100% 0:00:… 0:00:…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ----- 100% 0:00:… 0:00:…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ----- 100% 0:00:… 0:00:…
└── create_base_text_units
⠦ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ----- 100% 0:00:… 0:00:…
└── create_base_text_units
⠦ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ----- 100% 0:00:… 0:00:…
└── create_base_text_units
⠦ GraphRAG Indexer 
├── Loading 

In [9]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

c:\Users\15011\AppData\Local\anaconda3\envs\graphRagEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [17]:
INPUT_DIR = "./ragtest/output/20240718-170802/artifacts" 
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [18]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 448


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,top_level_node_id,x,y
0,0,"""ISRAEL J. MENDEZ BERMUDEZ""","""PERSON""",Israel J. Mendez Bermudez is a key contributor...,"0d4251e7621b5ee86c1d96355c11f5b1,116ce94a9589f...",1,10,0,b45241d70f0e43fca764df95b2b81f77,10,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,"""ESTEFANIA RAMIREZ MARQUEZ""","""PERSON""",Estefania Ramirez Marquez is one of the author...,"0d4251e7621b5ee86c1d96355c11f5b1,116ce94a9589f...",1,10,1,4119fd06010c494caa07f439b333f4c5,10,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,"""SOFÍA C. AYALA RODRÍGUEZ""","""PERSON""",Sofía C. Ayala Rodríguez is a key contributor ...,"0d4251e7621b5ee86c1d96355c11f5b1,116ce94a9589f...",1,10,2,d3835bf3dda84ead99deadbeac5d0d7d,10,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,"""ARMANDO J. RUIZ-JUSTIZ""","""PERSON""",Armando J. Ruiz-Justiz is one of the authors i...,"0d4251e7621b5ee86c1d96355c11f5b1,116ce94a9589f...",1,10,3,077d2820ae1845bcbb1803379a3d1eae,10,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,"""EDUARDO J. RODRIGUEZ-GARCIA""","""PERSON""",Eduardo J. Rodriguez-Garcia is a key contribut...,"0d4251e7621b5ee86c1d96355c11f5b1,116ce94a9589f...",1,10,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,10,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


#### Read relationships

In [19]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 180


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,"""ISRAEL J. MENDEZ BERMUDEZ""","""BILATERAL ACUTE IRIS TRANSILLUMINATION SYNDROME""",1.0,"""Israel J. Mendez Bermudez is one of the autho...",[ac2cc9e8c1c50c25a02ede88d04d1c1e],482027a59f32484c9c44fd700615c1b6,0,10,17,27
1,"""ISRAEL J. MENDEZ BERMUDEZ""","""ARMANDO L. OLIVER""",3.0,Israel J. Mendez Bermudez and Armando L. Olive...,"[0d4251e7621b5ee86c1d96355c11f5b1, 116ce94a958...",de837ff3d626451282ff6ac77a82216d,1,10,12,22
2,"""ISRAEL J. MENDEZ BERMUDEZ""","""ESTEFANIA RAMIREZ MARQUEZ""",3.0,Israel J. Mendez Bermudez and Estefania Ramire...,"[0d4251e7621b5ee86c1d96355c11f5b1, 116ce94a958...",460295fed3ae4cd39f9f274cec9c2506,2,10,10,20
3,"""ISRAEL J. MENDEZ BERMUDEZ""","""SOFÍA C. AYALA RODRÍGUEZ""",3.0,Israel J. Mendez Bermudez and Sofía C. Ayala R...,"[0d4251e7621b5ee86c1d96355c11f5b1, 116ce94a958...",553b285bba60460ab1ed8341ae61282b,3,10,10,20
4,"""ISRAEL J. MENDEZ BERMUDEZ""","""ARMANDO J. RUIZ-JUSTIZ""",3.0,Israel J. Mendez Bermudez and Armando J. Ruiz-...,"[0d4251e7621b5ee86c1d96355c11f5b1, 116ce94a958...",cec95bf17e7e4c939b56c9c6f402a29f,4,10,10,20


In [ ]:
# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

# claims = read_indexer_covariates(covariate_df)

# print(f"Claim records: {len(claims)}")
# covariates = {"claims": claims}

#### Read community reports

In [20]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 16


c:\Users\15011\AppData\Local\anaconda3\envs\graphRagEnv\lib\site-packages\graphrag\query\indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
c:\Users\15011\AppData\Local\anaconda3\envs\graphRagEnv\lib\site-packages\graphrag\query\indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,14,# Moxifloxacin and BAIT Syndrome\n\nThe commun...,3,8.5,Moxifloxacin and BAIT Syndrome,The impact severity rating is high due to the ...,The community revolves around the antibiotic M...,"[{'explanation': 'Moxifloxacin, a systemic ant...","{\n ""title"": ""Moxifloxacin and BAIT Syndrom...",c3ef32ba-427d-4e37-ba55-baa946077075
1,15,# BAIT Syndrome and Moxifloxacin\n\nThe commun...,3,7.5,BAIT Syndrome and Moxifloxacin,The impact severity rating is high due to the ...,"The community revolves around BAIT Syndrome, a...",[{'explanation': 'BAIT Syndrome has been stron...,"{\n ""title"": ""BAIT Syndrome and Moxifloxaci...",46268831-3dfd-46e5-90e7-0c06dd6a6381
2,12,# BAIT Syndrome and Moxifloxacin\n\nThe commun...,2,7.5,BAIT Syndrome and Moxifloxacin,The impact severity rating is high due to the ...,"The community revolves around BAIT Syndrome, a...","[{'explanation': 'BAIT Syndrome, or Bilateral ...","{\n ""title"": ""BAIT Syndrome and Moxifloxaci...",6489e6e3-cbf0-4088-b8f8-e4a289cc260a
3,13,# BAIT Syndrome in Europe\n\nThe community rev...,2,7.5,BAIT Syndrome in Europe,The impact severity rating is high due to the ...,The community revolves around Bilateral Acute ...,[{'explanation': 'BAIT Syndrome has a signific...,"{\n ""title"": ""BAIT Syndrome in Europe"",\n ...",46a73942-4772-4f5f-a02a-6c7ed0e61ae2
4,10,# Cureus and Mendez Bermudez\n\nThe community ...,1,7.5,Cureus and Mendez Bermudez,The impact severity rating is high due to the ...,The community revolves around the Cureus medic...,[{'explanation': 'Cureus is a medical journal ...,"{\n ""title"": ""Cureus and Mendez Bermudez"",\...",45acb20d-9b77-487e-a505-90c77f284fca


#### Read text units

In [21]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 25


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,ac2cc9e8c1c50c25a02ede88d04d1c1e,Review began 10/11/2023 \nReview ended 10/16/2...,400,[050380f20fd02add0f760e6df590b8de],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[482027a59f32484c9c44fd700615c1b6, 6ea81acaf23..."
1,fae41b752237c62e62a40b5d7610bcb0,"of Puerto Rico School of Medicine, Medical Sc...",400,[050380f20fd02add0f760e6df590b8de],"[27f9fbe6ad8c4a8b9acee0d3596ed57c, 9646481f66c...","[38f51478f41f48db9bee570859b6f43e, 896d2a51e8d..."
2,7aabe2b008c7f4cca3f97886cea28ed3,edema and transillumination in the right temp...,400,[050380f20fd02add0f760e6df590b8de],"[04dbbb2283b845baaeac0eaf0c34c9da, 1943f245ee4...","[31a7e680c4d54101afe4c8d52d246913, 50ea7d3b696..."
3,cad180e0a6759116f1c71a4ff57db3bc,anterior uveitis since patients typically pre...,400,[050380f20fd02add0f760e6df590b8de],"[c9632a35146940c2a86167c7726d35e9, 254770028d7...","[6fb57f83baec45c9b30490ee991f433f, d203efdbfb2..."
4,7fe5e923541990389c8694539762a84f,case\nof BAIT syndrome in a Hispanic female p...,400,[050380f20fd02add0f760e6df590b8de],"[254770028d7a4fa9877da4ba0ad5ad21, 1943f245ee4...","[d203efdbfb2f4b2a899abfb31cf72e82, 6731a665561..."


In [22]:
embedding_model = "text-embedding-3-large"
api_key = "34a6e9e765d94d3c8a318337cbc122cd"
llm_model = "gpt-4o"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
    api_version="2024-02-15-preview",
    deployment_name="gpt-4o",
    api_base= "https://df-pocs-q1.openai.azure.com/"
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base="https://df-pocs-q1.openai.azure.com/",
    api_type=OpenaiApiType.AzureOpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
    api_version="2024-02-01",
)

### Create local search context builder

In [23]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [24]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [25]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [27]:
question = "what is the country of the primary author? -please give in single word "
result = await search_engine.asearch(question)
print(result.response)

USA


#### Inspecting the context data used to generate the response

In [ ]:
result.context_data["entities"].head()

In [ ]:
result.context_data["relationships"].head()

In [ ]:
result.context_data["reports"].head()

In [ ]:
result.context_data["sources"].head()

In [ ]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [ ]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [ ]:
question_history = [
    "Tell me about Agent Mercer",
    "What happens in Dulce military base?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)